In [41]:
from pathlib import Path
from openai import OpenAI
import os
from dotenv import load_dotenv
from datetime import datetime

dotenv_path = Path("__file__").resolve().parents[1].parents[0] / '.local.env'
load_dotenv(dotenv_path)

OpenAI.api_key = os.getenv("OPENAI_API_KEY")
data_path = Path("__file__").resolve().parents[1].parents[0] / "local_data"

In [9]:
client = OpenAI()

file = client.files.create(
  file=open(os.path.join(data_path, 'data.jsonl'), "rb"),
  purpose="fine-tune"
)
file_id = file.id

FileObject(id='file-oInonTczvxFXMH1od8U4SblG', bytes=8717, created_at=1709010628, filename='data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [28]:
fine_tune = client.fine_tuning.jobs.create(
  training_file = file_id, 
  model = "gpt-3.5-turbo"
)
ftjob_id = fine_tune.id

FineTuningJob(id='ftjob-OkgVLbInsBp9X84afuSx0kZn', created_at=1709012610, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-slBL62vRUI2bYXEIDBVW0AEp', result_files=[], status='validating_files', trained_tokens=None, training_file='file-oInonTczvxFXMH1od8U4SblG', validation_file=None, user_provided_suffix=None)

In [34]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ftjob_id)

# Cancel a job
# client.fine_tuning.jobs.cancel("ftjob-H4bb8xeuH5UnbSmTfakdsrZN")

# List up to 10 events from a fine-tuning job
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-H4bb8xeuH5UnbSmTfakdsrZN", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
# client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:" + ftjob_suffix)

ftjob-OkgVLbInsBp9X84afuSx0kZn


FineTuningJob(id='ftjob-OkgVLbInsBp9X84afuSx0kZn', created_at=1709012610, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0613:personal::8wkdWCsh', finished_at=1709012985, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-slBL62vRUI2bYXEIDBVW0AEp', result_files=['file-efcXU8pQj2tuMhZM73GHWRpt'], status='succeeded', trained_tokens=13060, training_file='file-oInonTczvxFXMH1od8U4SblG', validation_file=None, user_provided_suffix=None)

In [43]:
# Check all models
ft_jobs = client.fine_tuning.jobs.list(limit=10)
for ft_job in ft_jobs:
  id = ft_job.id

  timestamp = ft_job.created_at
  datetime = datetime.fromtimestamp(timestamp)
  
  # State of fine tuning jobs
  state = client.fine_tuning.jobs.retrieve(id).status
  model = client.fine_tuning.jobs.retrieve(id).fine_tuned_model

  print(f'Create At: {datetime}')
  print(f'FineTune ID: {id}')
  print(f'Model ID: {model}')
  print(f'State: {state}\n')
  

Create At: 2024-02-27 14:43:30
FineTune ID: ftjob-OkgVLbInsBp9X84afuSx0kZn
Model ID: ft:gpt-3.5-turbo-0613:personal::8wkdWCsh
State: succeeded

Create At: 2024-02-27 14:14:23
FineTune ID: ftjob-EdiIsxlxDodSJcFAjS3ydNXC
Model ID: ft:gpt-3.5-turbo-0613:personal::8wkBcKAU
State: succeeded

Create At: 2024-02-27 14:10:38
FineTune ID: ftjob-H4bb8xeuH5UnbSmTfakdsrZN
Model ID: None
State: cancelled



In [47]:
response = completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8wkdWCsh", # Model ID
  messages=[
    {"role": "system", "content": "あなたは岸田文雄さんです。日本の政治家です。"},
    {"role": "user", "content": "減税について検討できますか？"}
  ]
)

In [48]:
print(response.choices[0].message)


ChatCompletionMessage(content='できます。', role='assistant', function_call=None, tool_calls=None)
